In [49]:
import pandas as pd
import numpy as np
from prince import FAMD, MCA
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

In [14]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [15]:
# Read in DataFrame
my_list = list(range(100))
for x in tqdm(my_list):
    df = pd.read_csv('../NYSDOH_BRFSS_SurveyData_2020.csv', encoding = 'cp1252')

<ipython-input-15-062ccc2311b6>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm(my_list):


  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (39,151,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# View first 5 rows of the data

df.head(5)

,_STATE,IDATE,IMONTH,IDAY,IYEAR,SEQNO,_PSU,COLGSEX,LANDSEX,RESPSLCT,...,PPS_40,PPS_43,PPS_44,PPS_45,PPS_46,PPS_48,PPS_52,SEXNOCON_OTH,NOVEGFRU_OTH,CHILDREN
0,New York,1152020,1,15,2020,2020002528,2020002528,Not asked or Missing,Not asked or Missing,Male,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
1,New York,1302020,1,30,2020,2020002529,2020002529,Not asked or Missing,Not asked or Missing,Male,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
2,New York,1152020,1,15,2020,2020002530,2020002530,Not asked or Missing,Female,Not asked or Missing,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
3,New York,2032020,2,3,2020,2020004509,2020004509,Not asked or Missing,Not asked or Missing,Female,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
4,New York,2152020,2,15,2020,2020002531,2020002531,Not asked or Missing,Not asked or Missing,Female,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing


In [16]:
# Use df_clean for cleaning
df_clean = df.copy()

# Repalce 'Not asked or Missing' and 'Data do not meet the criteria for statistical reliability, 
# data quality or confidentiality (data are suppressed)' with NA
for col in df.columns:
    df_clean[col].replace({'Not asked or Missing' : np.nan}, inplace = True)
    df_clean[col].replace({'Data do not meet the criteria for statistical reliability, data quality or confidentiality (data are suppressed)' : np.nan}, inplace = True)
    
# Drop columns with over 80% missing values
df_clean.dropna(axis = 1, thresh = len(df_clean) * .50, inplace = True)

In [25]:
# View non-null count 
df_clean.info()
print(len(df_clean))
print(df_clean['CVDCRHD4'].unique())
df_clean['CVDCRHD4'].replace('Don’t know/Not sure', np.nan, inplace=True)
df_clean['CVDCRHD4'].replace('Refused', np.nan, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14769 entries, 0 to 14768
Columns: 164 entries, _STATE to DSRIPREG
dtypes: float64(4), int64(10), object(150)
memory usage: 18.5+ MB
14769
['No' 'Yes' 'Don’t know/Not sure' nan]


In [28]:
print(df_clean['CVDCRHD4'].unique())
df_clean.dropna(subset=['CVDCRHD4'], inplace=True)

['No' 'Yes']


In [6]:
from imblearn.over_sampling import SMOTENC
smote_nc = SMOTENC()
X_resampled, y_resampled = smote_nc.fit_resample(X, y)
X = df_clean.loc[:, df_clean.columns != 'CVDCRHD4']
y = df_clean['CVDCRHD4']
oversample = SMOTE()
X, y = smote_nc.fit_resample(X, y)

TypeError: __init__() missing 1 required positional argument: 'categorical_features'

In [81]:
df_clean.head()
print(len(df_clean[df_clean['CVDCRHD4'] == 'No']))

33447


In [29]:
# Encode categorical values to get correlation matrix
df_clean_categorical = df_clean.copy()
cols = list(df_clean_categorical.columns)
for col in cols:
    if str(df_clean_categorical[col].dtype) == 'object':
        df_clean_categorical[col] = df_clean_categorical[col].astype('category').cat.codes

df_clean.head()

,_STATE,IDATE,IMONTH,IDAY,IYEAR,SEQNO,_PSU,CELLSEX,SEXVAR,GENHLTH,...,WTRSOURCE,STRSMEAL,FRUITVEG,MJUSE30,MJNYSMMP,HEPCTEST,HEPCTOLD,_WT2SPLITS,REGION,DSRIPREG
0,New York,1152020,1,15,2020,2020002528,2020002528,NaN,Male,Very good,...,Public Water Supply,Always,NaN,None,No,No,No,5130.843243,NYS exclusive of NYC,Long Island
1,New York,1302020,1,30,2020,2020002529,2020002529,NaN,Male,Very good,...,Public Water Supply,Never,NaN,None,No,No,No,941.550458,NYS exclusive of NYC,Long Island
2,New York,1152020,1,15,2020,2020002530,2020002530,NaN,Female,Good,...,Public Water Supply,Never,NaN,None,No,No,No,3497.366203,NYS exclusive of NYC,Long Island
3,New York,2032020,2,3,2020,2020004509,2020004509,NaN,Female,Good,...,Public Water Supply,Never,Neighborhood,None,No,NaN,NaN,1187.709030,NYS exclusive of NYC,Long Island
4,New York,2152020,2,15,2020,2020002531,2020002531,NaN,Male,Fair,...,Don't Know/Not Sure,Rarely,NaN,None,No,No,No,13364.387863,NYS exclusive of NYC,Long Island


In [30]:
df_clean_categorical_nonans = df_clean_categorical.apply(lambda x:x.fillna(x.value_counts().index[0]))
X = df_clean_categorical_nonans.loc[:, df_clean.columns != 'CVDCRHD4']
y = df_clean_categorical_nonans['CVDCRHD4']
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [35]:
print(Counter(y))
df_smote = pd.DataFrame(X)
df_smote['CVDCRHD4'] = y
df_smote.head()

Counter({0: 13950, 1: 13950})


,_STATE,IDATE,IMONTH,IDAY,IYEAR,SEQNO,_PSU,CELLSEX,SEXVAR,GENHLTH,...,STRSMEAL,FRUITVEG,MJUSE30,MJNYSMMP,HEPCTEST,HEPCTOLD,_WT2SPLITS,REGION,DSRIPREG,CVDCRHD4
0,0,1152020,1,15,2020,2020002528,2020002528,-1,1,6,...,0,-1,1,1,1,1,5130.843243,0,3,0
1,0,1302020,1,30,2020,2020002529,2020002529,-1,1,6,...,2,-1,1,1,1,1,941.550458,0,3,0
2,0,1152020,1,15,2020,2020002530,2020002530,-1,0,3,...,2,-1,1,1,1,1,3497.366203,0,3,0
3,0,2032020,2,3,2020,2020004509,2020004509,-1,0,3,...,2,1,1,1,-1,-1,1187.709030,0,3,0
4,0,2152020,2,15,2020,2020002531,2020002531,-1,1,2,...,4,-1,1,1,1,1,13364.387863,0,3,0


In [36]:
'''
import seaborn as sns

import matplotlib.pyplot as plt

# taking all rows but only 6 columns
df_small = df.iloc[:,:6]

correlation_mat = df_small.corr()

sns.heatmap(correlation_mat, annot = True)

plt.show()
'''
df_clean_corr = df_smote.corrwith(df_smote["CVDCRHD4"])

In [37]:
df_clean_corr_abs = df_clean_corr.abs()
df_clean_corr_abs.sort_values(inplace=True, ascending=False)
df_clean_corr_abs

CVDCRHD4    1.000000
_MICHD      0.969759
_RFHLTH     0.537006
_DRDXAR2    0.469149
CVDINFR4    0.445982
              ...   
_HISPANC    0.003010
MARITAL     0.001051
ECIGARET    0.000813
HISPANC3    0.000675
_STATE           NaN
Length: 164, dtype: float64

In [50]:
#Only use the top 30 columns, excluding columns like _MICHD, CVDINFR4, which ask about heart attack and coronary disease which is very close to heart disease.
feature_list = list(df_clean_corr_abs[3:35].keys())
feature_list.append('CVDCRHD4')
feature_list.remove('CVDINFR4')
feature_list

['_DRDXAR2',
 '_AGE80',
 '_HCVU651',
 '_EXTETH3',
 '_AGEG5YR',
 '_AGE_G',
 '_IMPAGE',
 '_STOLDNA',
 '_RFBLDS4',
 'STOOLDNA',
 'AGE',
 'COLNSCPY',
 'SHINGLE2',
 '_PHYS14D',
 'BLDSTOL1',
 '_SGMS10Y',
 'VIRCOLON',
 'REGION',
 '_SGMSCPY',
 'EMPLOY1',
 '_PNEUMO3',
 '_RACEG21',
 'MEDICARE',
 '_CRCREC1',
 '_SBONTIM',
 'RMVTETH4',
 '_FLSHOT7',
 '_VIRCOLN',
 'SIGMSCPY',
 '_IMPHOME',
 '_RFPAP35',
 'CVDCRHD4']

In [44]:
df_clean_columns = df_clean[feature_list]
df_clean_columns.head()

,_DRDXAR2,_AGE80,_HCVU651,_EXTETH3,_AGEG5YR,_AGE_G,_IMPAGE,_STOLDNA,_RFBLDS4,STOOLDNA,...,DRNKANY5,_DRNKWK1,_BMI5,DIABETE4,_ASTHMS1,DIFFWALK,MENTHLTH,_DRNKDRV,CNCRAGE,CVDCRHD4
0,Not diagnosed with arthritis,Imputed Age 65 to 69,"Don’t know/Not Sure, Refused or Missing",At risk,Age 65 to 69,Age 65 or older,Age 65 or older,Have never had a stool DNA test,Have never had a blood stool test,No,...,Yes,Number of drinks per week,1 or greater,No,Never,No,None,Have not driven after having too much to drink,Missing,No
1,Not diagnosed with arthritis,Imputed Age 55 to 59,Have health care coverage,Not at risk,Age 55 to 59,Age 55 to 64,Age 55 to 64,Have never had a stool DNA test,Have never had a blood stool test,No,...,Yes,Number of drinks per week,1 or greater,No,Never,No,None,Have not driven after having too much to drink,Missing,No
2,Diagnosed with arthritis,Imputed Age 80 or older,"Don’t know/Not Sure, Refused or Missing",At risk,Age 80 or older,Age 65 or older,Age 65 or older,"Missing, Age less than 50, Age greater than 75","Missing, Age less than 50, Age greater than 75",No,...,No,Did not drink,1 or greater,Yes,Never,Yes,None,Don’t know/Not Sure/Refused/Missing,Missing,No
3,Diagnosed with arthritis,Imputed Age 80 or older,"Don’t know/Not Sure, Refused or Missing",Not at risk,Age 80 or older,Age 65 or older,Age 65 or older,"Missing, Age less than 50, Age greater than 75","Missing, Age less than 50, Age greater than 75",No,...,Yes,Number of drinks per week,1 or greater,No,Never,Yes,Don’t know/Not sure,Have not driven after having too much to drink,Missing,No
4,Not diagnosed with arthritis,Imputed Age 40 to 44,"Don’t know/Not Sure, Refused or Missing",Not at risk,Age 40 to 44,Age 35 to 44,Age 35 to 44,"Missing, Age less than 50, Age greater than 75","Missing, Age less than 50, Age greater than 75",NaN,...,No,Did not drink,1 or greater,Yes,Never,No,Number of days,Don’t know/Not Sure/Refused/Missing,Missing,No


In [51]:
from sklearn.model_selection import train_test_split 

# Drop all missing values
df_cleaned = df_clean_columns.dropna(axis = 0).reset_index(drop = True)
print(df_cleaned['CVDCRHD4'])

# Drop all rows that are Don't know/Not sure or Refused for column we are predicting
df_cleaned = df_cleaned.loc[(df_cleaned['CVDCRHD4'] == 'No') | (df_cleaned['CVDCRHD4'] == 'Yes')]

# Split into X and y
X = df_cleaned.loc[:, df_cleaned.columns != 'CVDCRHD4']
y = df_cleaned['CVDCRHD4']

# Split the data into training and test data set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3,random_state = 42)

0       No
1       No
2       No
3       No
4       No
        ..
7138    No
7139    No
7140    No
7141    No
7142    No
Name: CVDCRHD4, Length: 7143, dtype: object


In [ ]:
null_vals = X_train.isnull().sum()
null_vals_desc = null_vals.sort_values(ascending=False)
null_vals_desc

In [52]:
'''famd = FAMD(n_components = 70, n_iter = 3, random_state = 42)
X_train_transformed = famd.fit_transform(X_train)
X_train_transformed.head()'''

mca = MCA( n_components=2, n_iter=3, copy=True, check_input=True,engine='auto', random_state=42)
mca = mca.fit(X)

In [53]:
RANDOM_SEED = 694

#Run dummy classifier to find the baseline performance
dummy_clf = DummyClassifier(strategy= 'most_frequent').fit(mca.fit_transform(X_train),y_train)
y_pred = dummy_clf.predict(mca.fit_transform(X_test))

In [48]:
#helper function
def get_performance_scores(y_pred, y_true):
    f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    return [f1, accuracy, precision, recall]

def print_performance_scores(scores):
    print("Accuracy Score = " + str(scores[1]))
    print("Precision Score = " + str(scores[2]))
    print("Recall Score = " + str(scores[3]))
    print("F1 Score = " + str(scores[0]))

In [54]:
#Find the performance of the model
dummy_f1 = f1_score(y_test, y_pred, average='macro')
dummy_accuracy = accuracy_score(y_test, y_pred)
dummy_precision = precision_score(y_test, y_pred, average='macro')
dummy_recall = recall_score(y_test, y_pred, average='macro')
print("Accuracy Score = " + str(dummy_accuracy))
print("Precision Score = " + str(dummy_precision))
print("Recall Score = " + str(dummy_recall))
print("F1 Score = " + str(dummy_f1))

Accuracy Score = 0.9160055996266916
Precision Score = 0.4580027998133458
Recall Score = 0.5
F1 Score = 0.4780808572820263


C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
clf_lr = LogisticRegression(random_state = RANDOM_SEED).fit(X_train_transformed, y_train)

In [77]:
X_test_transformed = famd.fit_transform(X_test)
train_preds = clf_lr.predict(X_test_transformed)

In [ ]:
train_preds

In [78]:
lr_f1 = f1_score(y_test, train_preds, average='macro')
lr_accuracy = accuracy_score(y_test, train_preds)
lr_precision = precision_score(y_test, train_preds, average='macro')
lr_recall = recall_score(y_test, train_preds, average='macro')
print("Accuracy Score = " + str(lr_accuracy))
print("Precision Score = " + str(lr_precision))
print("Recall Score = " + str(lr_recall))
print("F1 Score = " + str(lr_f1))

Accuracy Score = 0.9105705598282955
Precision Score = 0.5276774507543738
Recall Score = 0.5008645763735802
F1 Score = 0.4805556298813463


In [79]:
#Do grid search for hyperparameter tuning
clf = LogisticRegression(random_state = RANDOM_SEED, solver='liblinear')
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values,scoring = 'f1_macro')
my_list = list(range(100))
for x in tqdm(my_list):
    grid_clf_acc.fit(X_train_transformed, y_train)

y_pred_acc = grid_clf_acc.predict(X_test_transformed)

print_performance_scores(get_performance_scores(y_pred_acc, y_test))

<ipython-input-79-b8f27912ffa5>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm(my_list):


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf_clf = RandomForestClassifier(random_state=RANDOM_SEED).fit(X_train_transformed, y_train)

grid_values = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
 }

rand_search_clf = RandomizedSearchCV(estimator = rf_clf, param_distributions = grid_values, n_iter = 1, cv = 3, verbose=2, random_state=RANDOM_SEED, n_jobs = -1)

my_list = list(range(100))
for x in tqdm(my_list):
    rand_search_clf.fit(X_train_transformed[0:10000], y_train[0:10000])


y_pred_rf = rand_search_clf.predict(X_test_transformed)
   


print_performance_scores(get_performance_scores(y_pred_rf, y_test))

In [54]:
random_forest = RandomForestClassifier(n_estimators = 100, random_state = RANDOM_SEED )
random_forest.fit(X_train_transformed, y_train)

y_pred = random_forest.predict(X_test_transformed)

print_performance_scores(get_performance_scores(y_pred, y_test))

Accuracy Score = 0.9077134986225895
Precision Score = 0.6428324099722992
Recall Score = 0.509469696969697
F1 Score = 0.4971883398800909
